# Face Recognition

One of the most important application used today of Computer Vision is Face Recognition, In this Notebook i will present two main applications of Face Recognition and build Face Unlock Systems. Many of the ideas presented here are from [FaceNet](https://arxiv.org/pdf/1503.03832.pdf) and from the Deep Learning Specilization 

Face recognition problems commonly fall into two categories: 

- **Face Verification** - "is this the claimed person?". For example, at some airports, you can pass through customs by letting a system scan your passport and then verifying that you (the person carrying the passport) are the correct person. A mobile phone that unlocks using your face is also using face verification. This is a 1:1 matching problem. 
- **Face Recognition** - "who is this person?". For example,an employee entering the office the system checkes that his face is in the database and he has access to pass and then opens the door for him This is a 1:K matching problem. 

## Face Verification

##### 1 - Smartphones Face Unlock

- Most smartphones today use face verification to unlock the smartphone, when you setup face unlock on your phone the front facing camera takes a picture of your face and uses a Convolution neural network to learn an encoding vector for your face and saves this encoding in a database 

- So when someone tries to unlock your phone again the front faceing camera takes a new picture of the face it sees at that moment, computes an encoding for that picture and compare it with the previous encoding saved in the database if it's you the encodings will matche and unlocks your phone if not it keeps the phone locked

#####  This system will be implemented in this notebook using Inception ConvNet Structure

- The inception network uses a series of inception blocks as shown in figure 1
- This network is implmented in keras in inception_blocks.py file and imported in this notebook checkout the implemntation if interested
    
<img src="images/inception_module.png" style="width:500px;height:200px;">
<caption><center> <u> <font color='purple'> Figure 1 </u></center></caption>


In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


Let's Call our Model from inception_blocks.py

- This network uses 96x96 dimensional RGB images as its input. Specifically, inputs a face image (or batch of $m$ face images) as a tensor of shape $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$ 
- It outputs a matrix of shape $(m, 128)$ that encodes each input face image into a 128-dimensional vector

In [2]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96));
print("Total Params:", FRmodel.count_params())

Instructions for updating:
Colocations handled automatically by placer.
Total Params: 3743280


### The Triplet Loss

To train the Model we will use the Triplet Loss Function as measure for how good the model does

Training will use triplets of images $(A, P, N)$:  

- A is an "Anchor" image--a picture of a person. 
- P is a "Positive" image--a picture of the same person as the Anchor image.
- N is a "Negative" image--a picture of a different person than the Anchor image.

These triplets are picked from our training dataset. We will write $(A^{(i)}, P^{(i)}, N^{(i)})$ to denote the $i$-th training example. 

If our Phone unlock system has an Image A of a Person in the setup and took a new picture of the Person P when trying to unlock the phone We'd like to train the model to make sure that an image $A^{(i)}$ of an individual is closer and similar to the Positive image $P^{(i)}$ than to the Negative image $N^{(i)}$) by at least a margin $\alpha$:

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

So The Model will minimize the following "triplet cost":

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{1}$$


In [3]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss Cost as defined in (1)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor , positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    #measures how similar is the Anchor image saved in the database to a positive image of a person
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)), axis=-1, keepdims=True)
    #measures how similar is the anchor image to a negative image of different person
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)), axis=-1, keepdims=True)
    
    basic_loss = pos_dist + alpha - neg_dist
    
    loss = tf.reduce_sum(tf.maximum(basic_loss,0)) # The Final Cost J
    
    return loss

The Model is trained by minimizing this triplet loss, Since Training requires a lot of data and a lot of computation and time it's common practice in deep learning to use pre-trained weights and do Transfer-Learning, when building our system we don't want the user to experinece this training computation cost to setup face-unlock so we will use pre-trained weights.

We train a model for face-unlock on a very large dataset of faces to minimize the triplet loss on different machine that handles the computation cost and save the model weights and load it on the smartphone thus we already have neural network that know how to learn from faces, when a user setup the face-unlock we might further-train on images of his face if we want to further optimize our algorithim 

so in the next cell i load pre-trained weights on a very large datasets of faces and we will use it to further train the model 

We will use adam Algorithim for optimization and the Triplet loss function we defined and accuracy as measure 

In [4]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

Next We will define a function that maps images into encodings using our defined model

In [5]:
def img_to_encoding(image_path, model):
    img1 = cv2.imread(image_path, 1)
    img = img1[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

#### Face Unlock
Now let's Build our Face Unlock System

Younes tried To setup face-unlock on his Smartphone for the first time 

- The phone took the following picture of younes face 

<img src="images/younes.jpg" style="width:100px;height:100px;">

- for this picture in the next cell the algorithim will compute an encoding vector for younes face we call it True_encoding
- we will save this encoding in a database


In [6]:
database = {}

database["younes"] = img_to_encoding("images/younes.jpg", FRmodel) #true Encoding

Now When someone pickup the Phone

- The Camera will take a picture of the face it sees and saves it in a directory img_path
- We will load that picture from that path (img_path) and compute an encoding for it using our model we will call it new_encoding
- Next We will Compare this New_endcoding with The True_encoding we have in our database 
- If the two encodings are similar and the distance is less then some threshold for example 0.7 the phone will unlock
- if the two encoding are less similar and the distance are higher than this threshold the phone will keep locked

let's implement the following system

In [7]:
def face_unlock(image_path,database,model):
    """
    Arguments:
    image_path -- path to an image
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model --  Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    phone_unlock -- True, if the Phone should unlock. False otherwise.
    """
    
    #considring we setup the phone with some code to take pictures and saves them in a directory img_path
    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm(np.subtract(encoding,database["younes"]),ord=2)   
    
    if dist < 0.7:
        print('Phone Unlocked!')
        phone_unlock = True
    else:
        print('Only younes is Allowed to Unlock The Phone')
        phone_unlock = False
        
    return dist,phone_unlock

Someone picked up the phone and tried to unlock it, the front faceing camera took the following picture for him, the picture was saved in the following directory "images/camera_0.jpg" let's see what our model will do

<img src="images/camera_0.jpg" style="width:100px;height:100px;">

In [8]:
face_unlock("images/camera_0.jpg", database, FRmodel)

Phone Unlocked!


(0.65938383, True)

Someone picked up the phone and tried to unlock it, the front faceing camera took the following picture for him, the picture was saved in the following directory  "images/camera_2.jpg" let's see what our model will do

<img src="images/camera_2.jpg" style="width:100px;height:100px;">

In [9]:
face_unlock("images/camera_2.jpg", database, FRmodel)

Only younes is Allowed to Unlock The Phone


(1.0097818, False)

### 2 - Face Recognition

Some company wants to build Face-Recongition System that only authorized employees can enter the building, one way is to use id card for each employee, an employee swips his id at the door and it opens for him

Deep learning allows us to get rid of id-cards and use more secure systems because id-cards could be stolen so we will build a face-recognition system, we will use the same idea used to build the phone-unlock and use the same network

Setting up:
- when setting up the system for the first time the camera will take a picture for each authorized employee.
- the model will compute an ecnoding for each picture for each employee using img_to_encoding()
- the encodings will be saved in a database each employee name with his corrsponding encoding
- when someone shows up at the door and tries to enter the building the system will take a new picture for him and saves it in directory img_path
- the system will load that picture from img_path and compute an encoding for it 
- we will compare this new_encoding with all of the encodings we have in the database and see which one is the closest to it and calculate the distance 
- we will check if the distance is less than some threshold for example 0.7 to make sure if so then unlock the door, if it's higher than this person is not in the database of authorized persons

let's create a database for all the authorized employees
a picture is taken for each person with the camera and using our model we will compute an encoding for it and save it in databae

In [10]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

In [15]:
def face_recognition(img_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
        
    encoding = img_to_encoding(img_path, model)
    min_dist = 100
    
    for (name,enc) in database.items():
        dist = np.linalg.norm(encoding - enc,ord=2)
        if dist < min_dist:
            min_dist = dist
            identity = name
    if min_dist < 0.7:
        print("door unlocked it's {}".format(identity))
    else:
        print("You are not Authorized to enter")
        identity = 'Unknown'
        
    return min_dist,identity
        

This All The images of the authorized persons taken in the setup and we have their encoding saved in the database

<table><tr><td><img src='images/danielle.png'></td><td><img src='images/benoit.jpg'></td><td><img src='images/tian.jpg'></td></tr></table>
<table><tr><td><img src='images/arnaud.jpg'></td><td><img src='images/kian.jpg'></td><td><img src='images/dan.jpg'></td></tr></table>
<table><tr><td><img src='images/sebastiano.jpg'></td><td><img src='images/bertrand.jpg'></td><td><img src='images/kevin.jpg'></td></tr></table>

Someone showed up at the door and tried to enter the building, the camera took the following picture for him, the picture was saved in the following directory  "images/camera_1.jpg" let's see what our model will do

<img src="images/camera_1.jpg" style="width:100px;height:100px;">

In [12]:
face_recognition("images/camera_1.jpg", database, FRmodel)

door unlocked it's bertrand


(0.46768862, 'bertrand')

Someone showed up at the door and tried to enter the building, the camera took the following picture for him, the picture was saved in the following directory  "images/camera_4.jpg" let's see what our model will do

<img src="images/camera_4.jpg" style="width:100px;height:100px;">

In [21]:
face_recognition("images/camera_4.jpg", database, FRmodel)

door unlocked it's dan


(0.25046661, 'dan')

Someone showed up at the door and tried to enter the building, the camera took the following picture for him, the picture was saved in the following directory  "images/camera_0.jpg" let's see what our model will do

<img src="images/camera_0.jpg" style="width:100px;height:100px;">

In [16]:
face_recognition("images/camera_0.jpg", database, FRmodel)

You are not Authorized to enter


(0.7455134, 'Unknown')